In [202]:
import os
import yaml
import pandas as pd
import scipy
from scipy.stats import chi2
from scipy.stats import norm
from scipy.stats import sem, t
from scipy.stats import linregress
from scipy.stats import normaltest
from scipy.stats import chi2
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sns.set()
import json

import glob
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import axes3d
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
%matplotlib notebook

In [203]:
df = pd.read_csv("argus_info_output_fluidics.csv")
df['added'] = False

In [204]:
df.head()

,timestamp,layer,parent,has_doughter,duration,incubation_duration,type,end_flag,extended_description,doughter_duration,execution_time,added
0,2022-11-22 17:27:43.648,0,NaN,True,150.585,60.0,HURWellExchange,end,"{'selected_wells': [{'carrier_id': 'unit0', 's...",89.874,0.711,False
1,2022-11-22 17:27:44.267,1,0.0,True,84.486,0.0,HURDispense,end,"{'selected_wells': [{'carrier_id': 'unit0', 's...",84.356,0.130,False
2,2022-11-22 17:27:44.271,2,1.0,True,30.778,0.0,WellExtract,end,"{'selected_wells': [{'carrier_id': 'unit0', 's...",25.074,5.704,False
3,2022-11-22 17:27:44.378,3,2.0,True,2.574,0.0,SetADPZToSafeTravelHeight,end,{'entry_criteria': [<argus.fluidics.cs1.fl_adp...,2.567,0.007,False
4,2022-11-22 17:27:44.382,4,3.0,False,2.567,0.0,SetADPZPosition,end,"{'z_target_nm': 100000, 'acceleration_nmps': 1...",0.000,2.567,False


In [205]:
def add_layer_to_dict(current_dict, lineage_array, lineage_time, lineage_added, lineage_sum_time):
    current_layer = lineage_array.pop(0)
    current_time = lineage_time.pop(0)
    current_added = lineage_added.pop(0)
    current_sum_time = lineage_sum_time.pop(0)
    
    if current_dict.get(current_layer) != None:
        if current_added == False:
            current_dict[current_layer]['times'].append(current_time)
            current_dict[current_layer]['sum_times'].append(current_sum_time)
    else:
#         print("adding {} to {}".format(current_layer, lineage_array))
        current_dict[current_layer] = {'times': [], 'sum_times': [], 'substeps': {}}
        current_dict[current_layer]['times'].append(current_time)
        current_dict[current_layer]['sum_times'].append(current_sum_time)
        
    if len(lineage_array) > 0:
        add_layer_to_dict(current_dict[current_layer]['substeps'], lineage_array, lineage_time, lineage_added, lineage_sum_time)
    else:
        return
    
    

In [206]:
top_level = {}
for i, row in enumerate(df.iterrows()):
    row = row[1]
    layer = row['layer']
            
    if row['has_doughter'] == False:
        step_type = row['type']
        layer = row['layer']
        lineage_array = []
        lineage_time = []
        lineage_sum_time = []
        lineage_added = []
        lineage_array.append(row['type'])
        lineage_time.append(row['execution_time'])
        lineage_sum_time.append(row['doughter_duration'])
        lineage_added.append(row['added'])
        df.iloc[[i],[11]] = True
        parent_row_iloc = int(row['parent'])
        while layer != 0:
            parent_type = df.iloc[parent_row_iloc]['type']
            layer = df.iloc[parent_row_iloc]['layer']
            exec_time = df.iloc[parent_row_iloc]['execution_time']
            daughter_time = df.iloc[parent_row_iloc]['doughter_duration']
            added_state = df.iloc[parent_row_iloc]['added']
            df.iloc[[parent_row_iloc],[11]] = True
            try:
                parent_row_iloc = int(df.iloc[parent_row_iloc]['parent'])
            except ValueError:
                pass #Reached a NaN from top layer
            lineage_array.insert(0, parent_type)
            lineage_time.insert(0, exec_time)
            lineage_added.insert(0, added_state)
            lineage_sum_time.insert(0, daughter_time)
        
        add_layer_to_dict(top_level, lineage_array, lineage_time, lineage_added, lineage_sum_time)
df['added'] = False

In [207]:
def nested_dict_pairs_iterator(current_dict):
    try:
        current_dict['stats'] = []
    except TypeError:
        print("hit the type error")
        return
    
    keys = current_dict.keys()
    
    if 'substeps' in keys:
        current_dict['stats'].append(np.mean(current_dict['times'])) # step time mean
        current_dict['stats'].append(np.std(current_dict['times'])) # step time std
        current_dict['stats'].append(np.mean(current_dict['sum_times'])) # aggregate time mean
        current_dict['stats'].append(np.std(current_dict['sum_times'])) # aggregate time std

        keys = current_dict['substeps'].keys()
    
        if keys:
            print(keys)
            print("*"*80)
            for key in keys:
                nested_dict_pairs_iterator(current_dict['substeps'][key])
        else:
            return
    else:
        if keys:
            for key in keys:
                nested_dict_pairs_iterator(current_dict[key])
        else:
            return
    

In [208]:
nested_dict_pairs_iterator(top_level)

dict_keys(['HURDispense', 'AbsolutePositionSyringePumps', 'SetObjectiveZToSafeTravelHeight'])
********************************************************************************
dict_keys(['WellExtract', 'SetADPZToSafeTravelHeight', 'SetSipperZToSafeTravelHeight', 'SetObjectiveZToSafeTravelHeight', 'AlignHURNozzleToWell', 'DispenseSyringePumps', 'AbsolutePositionSyringePumps'])
********************************************************************************
dict_keys(['SetADPZToSafeTravelHeight', 'StartPeristalticPump', 'AlignSipperTubeToWellExtractLocation', 'MachineVisionCorrection', 'SetSipperZToSafeTravelHeight', 'StopPeristalticPump'])
********************************************************************************
dict_keys(['SetADPZPosition'])
********************************************************************************
dict_keys(['SetGantryXPosition', 'SetCarrierPosition', 'SetSipperZPosition'])
********************************************************************************
di

In [209]:
top_level

{'HURWellExchange': {'times': [0.7110000000000127,
   0.7110000000000127,
   0.6280000000000427,
   0.7109999999999559,
   0.7190000000000225,
   0.8200000000000003,
   5.571000000000005,
   2.415999999999997,
   2.512999999999998,
   2.5670000000000077,
   1.1809999999999974,
   3.0189999999999984,
   4.766999999999996,
   2.2820000000000107,
   0.7339999999999236,
   0.6889999999999645,
   0.7260000000000275,
   0.6899999999999693,
   1.3120000000000047,
   4.548000000000002,
   4.1510000000000105,
   0.7070000000000078,
   0.6959999999999127,
   0.728999999999985,
   0.7410000000000139,
   0.7629999999999768,
   1.1460000000000008,
   3.6260000000000048,
   2.9069999999999965,
   0.7000000000000028,
   0.8450000000000273,
   0.7599999999999909,
   0.8319999999999936,
   0.8829999999999814,
   1.227000000000004,
   3.6850000000000014,
   3.201999999999998,
   1.9599999999999935,
   0.7430000000000518,
   0.7170000000000414,
   0.7489999999999952,
   0.7369999999999663,
   1.297000000